pixyz  
last update 2022 03 23  
ゆっくりしていってね！  

version6: Items that costomer has bought are no longer included in the submission

**EDA書いたのでよかったら見てね。**

https://www.kaggle.com/lunapandachan/h-m-eda-english

**datasetをつくるnotebookも観てね！**  
https://www.kaggle.com/code/pixyz0130/eng-h-m-items-of-other-customers-buy

<img src="https://4.bp.blogspot.com/-6jLig_Zuhyk/UUhH8z560_I/AAAAAAAAO6A/lFCDFT8S1FM/s400/shopping_fasion.png" width=200>

**霊夢:今日は前のnotebookで作ったdatasetでsubmissionを作っていくよ‼**

**魔理沙:Byfone氏のcodeを参考にしたぜ。**

<br>

**Reimu: Today I'm going to make a submission with the dataset I made in the previous notebook!**

**Marisa: I referred to Mr. Byfone's code.**

https://www.kaggle.com/byfone/h-m-trending-products-weekly

<img src = "https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 200>

**霊夢:データが多いのでtest=Trueの時は1000個のデータを使うよ。**  

**Reimu: Since there is a lot of data, 1000 data will be used when test = True.**

In [ ]:
test=False

In [ ]:
import numpy as np
import pandas as pd
import json
from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

In [ ]:
data_path = Path('../input/h-and-m-personalized-fashion-recommendations/')
N = 12
M = 100

**魔理沙:N=12は予測アイテムの最大数だぜ。**

**Marisa: N = 12 is the maximum number of predictable items.**

### Read the train data
trainデータの読み込み

In [ ]:
df = pd.read_csv(data_path / 'transactions_train.csv',
                 usecols = ['t_dat', 'customer_id', 'article_id'],
                 dtype={'article_id': str})

if test :
    df=df[:1000]

### Take supposedly popular products

おそらく人気のある製品を取る

**魔理沙:predictionが12個に満たなかった時や、trainデータに無いcustomer_idがあったときのために、総購入数が多い商品のランキングを作るよ。**

**霊夢:実際にtraindataに無いcustomer_idが存在するとは思えないけどなあ...**

<br>

**Marisa: I'll make a ranking of the products with the highest total purchases in case the prediction is less than 12 or there is a customer_id that is not in the train data.**

**Reimu: I don't think there is actually a customer_id that isn't in traindata ...**

<img src="https://1.bp.blogspot.com/-_s9y-vGJIvM/Xhwql3w5nLI/AAAAAAABXBs/J1_f2TIYxukb1VxSlzn1xSysPsGd4lf3ACNcBGAsYHQ/s1600/pose_syanikamaeru_woman.png" width=200>

In [ ]:
general_pred = df["article_id"].astype(str).value_counts().sort_values(ascending=True)[-N:].index.to_list()

### Load json
jsonファイルを読み込む

**魔理沙:次は、前回つくったdatasetを読み込んでくよ！**

**霊夢:「ある商品を買った人が他に買った商品ランキング」のデータだったよね。実際に使えるかなあ**

＜br＞

**Marisa: Next, let's load the dataset we created last time!**

**Reimu: It was the data of "the ranking of products that the person who bought one product bought another". I wonder if it can actually be used**

In [ ]:
with open("../input/hm-dictionary/items_of_other_costomers.json", mode="r") as f:
        ds_dict = json.load(f)
with open("../input/hm-dictionary/dict_c_a.json", mode="r") as f:
        c_a_dict = json.load(f)        

In [ ]:
# for x in ds_dict:
#     ds_dict[x] = ds_dict[x][:12]

### Fill in purchase dictionary

購入辞書に記入する

**魔理沙:それじゃあ次は、それぞれの客が購入した商品から、次に買いそうな商品を予測していくよ**  

**霊夢:どうやって作るんだ？**  

**魔理沙:さっき読み込んだdatasetを使って、客が買ったそれぞれの商品について、その商品を買った時、他に買う商品ランキング1位はM点、2位はM-1点、3位はM-2点、4位はM-3点、という感じに配点をしていくぜ。その後、配点が高かった12個の商品をsubmissionにするぜ。これをすべての客について行うぜ。もしも、点数が1点以上の商品が12個よりも少なかったら、general_predから補充するぜ。**

**Marisa: Then, from the products purchased by each customer, we will predict the products that are likely to be bought next**

**Reimu: How do you make it?**

**Marisa: For each product that the customer bought using the dataset that was just read, when the customer bought that product, the other product rankings they bought were M points, 2nd place was M-1 points, and 3rd place was M-2 points. I will give points to the point, 4th place is M-3 point. After that, I will submit the 12 products with the highest points. I'll do this for every guest. If there are less than 12 items with 1 or more points, I will replenish them from general_pred.**

<img src="https://2.bp.blogspot.com/-GuJM5cIi3K8/VHbNOnnoEzI/AAAAAAAApU8/VAa2CK1C360/s400/hyousyou_sports_man.png" width=200>

In [ ]:
sub = pd.read_csv(data_path / 'sample_submission.csv')

pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in c_a_dict:
        purchase_dict = {}
        past_list = c_a_dict[cust_id]        
        for art_id in past_list:
            rank_list = ds_dict[str(art_id)]
            for j in range(M):
                item = str(rank_list[j]).zfill(10)
                if item not in purchase_dict:
                    purchase_dict[item] = M-j
                else:
                    purchase_dict[item] += M-j
        for art_id in past_list:
            if str(art_id).zfill(10) in purchase_dict:
                purchase_dict[str(art_id).zfill(10)] = 0
        series = pd.Series(purchase_dict)
        series = series[series > 0]
        
#         print(len(series))
        
        sub_list = series.nlargest(N).index.tolist()
    else:
        sub_list = general_pred
    pred_list.append(' '.join(sub_list))
sub['prediction'] = pred_list
sub.to_csv('submission.csv', index=None)

### Make a submission

In [ ]:
sub.head()

**霊夢:0706016001ってのがたくさんあるね。大丈夫かな？**

**魔理沙:確か最も多く買われてた商品だったな。一番上に来てもおかしくはないぞ。ほかの商品は散っているみたいだし、悪くないんじゃないか？**

<br>

**Reimu: There are a lot of 076016001. Is it OK?**

**Marisa: It was certainly the most bought item. It wouldn't be strange to come to the top. It seems that other products are scattered, isn't it bad?**

![](https://1.bp.blogspot.com/-PNcKwFw1PpM/U1T3oDIr9CI/AAAAAAAAfT4/gEn86X8Ppx0/s400/figure_goodjob.png)